In [ ]:
!pip install --upgrade pip
!pip install PyYaml
!pip install openai

In [ ]:
import yaml
import openai
from pprint import pprint
import json
import time

In [ ]:
!pwd

/datasets/_deepnote_work/openai-projects


In [ ]:
with open("configs/TJ_OPENAI_KEY.yaml", "r") as f:
    cx = yaml.safe_load(f)

In [ ]:
client = openai.OpenAI(
    # This is the default and can be omitted
    api_key=cx['openai_api_key'],
)

### Step 1: Ask for Company, Department and Designation

In [ ]:
print("Step 1: Enter the info of the company eg.. Financial Technology SaaS company")
company = input("> Enter here: ")
print("You have entered - {}\n\n--------------------\n".format(company))
print("\nStep 2: Enter your designation eg.. Engineering Manager")
designation = input("> Enter here: ")
print("You have entered - {}\n\n--------------------\n".format(designation))
print("\nStep 3: Enter your team eg.. Technology team")
department = input("> Enter here: ")
print("You have entered - {}\n\n--------------------\n".format(department))

Step 1: Enter the info of the company eg.. Financial Technology SaaS company
You have entered - Educational Software provider

--------------------


Step 2: Enter your designation eg.. Engineering Manager
You have entered - Engineering Manager

--------------------


Step 3: Enter your team eg.. Technology team
You have entered - Technology team

--------------------



### Step 2: Get 10 Questions for the Designation/Department/Company

In [ ]:
prompt = "What should be the 10 questions that you can ask to find out the skills that a {designation} in {department} has who is working in a {company}?".format(designation=designation, department=department, company=company)
print(prompt)

What should be the 10 questions that you can ask to find out the skills that a Engineering Manager in Technology team has who is working in a Educational Software provider?


In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
questions = chat_completion.choices[0].message.to_dict()['content']
print(questions)

1. Can you provide examples of your experience leading technology teams in the development and delivery of educational software products?
2. How do you stay current with emerging technologies and trends in the educational software industry?
3. How do you approach project management and ensure successful implementation of software development projects?
4. Can you describe a time when you had to resolve a technical issue or bottleneck within your team? How did you handle it?
5. How do you prioritize and allocate resources, such as budget and manpower, for technology projects within an educational software provider?
6. Can you give examples of how you have collaborated with cross-functional teams, such as product, design, and marketing, to achieve successful outcomes for educational software products?
7. How do you assess and address the technical skills and development needs of your team members?
8. Can you discuss a time when you successfully led a team through a major technology upgrad

### Step 3: Get Required Skills that the Designation/Department/Company

In [ ]:
prompt2 = "What should be skills that a {designation} in {department} should have who is working in a {company}? Please provide the category and skills inside that category".format(designation=designation, department=department, company=company)
print(prompt2)

What should be skills that a Engineering Manager in Technology team should have who is working in a Educational Software provider? Please provide the category and skills inside that category


In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt2,
        }
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
skills = chat_completion.choices[0].message.to_dict()['content']
print(skills)

1. Technical Skills:
   a. Proficiency in software development languages and frameworks
   b. Understanding of database management and design
   c. Knowledge of cloud computing technologies and platforms
   d. Ability to troubleshoot technical issues and provide solutions

2. Leadership Skills:
   a. Strong communication skills to effectively convey ideas and instructions to team members
   b. Ability to inspire and motivate team members to achieve common goals
   c. Decision-making and problem-solving abilities to resolve conflicts and make strategic decisions
   d. Project management skills to prioritize tasks and meet deadlines

3. Educational Domain Knowledge:
   a. Understanding of educational software development principles and practices
   b. Knowledge of learning management systems and educational technologies
   c. Familiarity with educational standards and regulations
   d. Ability to create innovative solutions that cater to the needs of educators and students

4. Team Build

### Step 4: Create instructions (prompt) for the assistant usint the generated questions

#### Main Context for the Assistant
```
## Aim: 
- Ask questions to gather information about the skills of the user  
- Use the provided questions as a guideline to ask questions

## Persona: 
- Always stay on point and do not get distracted
- Encourage user to think deeply and articulate their thoughts clearly
- Be encouraging and inquisitive
- Maintain professionalism in all circumstances
- Ensure the user feels supported throughout the conversation

## Restrictions
- Do not deviate from a professional demeanor
- Do not use slang, jargon, or inappropriate language under any circumstances
- Do not provide answers or opinions on the topics discussed
- Do not disclose or discuss any specific reasons behind asking the questions

## Ending the conversation
- End after all information is gathered
- End with this exact phrase "Thank you for your anwsers. Hope you have a good day. Bye."

```

#### Additional Context for each run
```
## Questions to use as guidelines
{{ Add the generated questions }}
```

In [ ]:
assistant_instructions = '''## Aim: 
- Ask questions to gather information about the skills of the user  
- Use the provided questions as a guideline to ask questions

## Persona: 
- Always stay on point and do not get distracted
- Encourage user to think deeply and articulate their thoughts clearly
- Be encouraging and inquisitive
- Maintain professionalism in all circumstances
- Ensure the user feels supported throughout the conversation

## Restrictions
- Do not deviate from a professional demeanor
- Do not use slang, jargon, or inappropriate language under any circumstances
- Do not provide answers or opinions on the topics discussed
- Do not disclose or discuss any specific reasons behind asking the questions

## Ending the conversation
- End after all information is gathered
- End with this exact phrase "Thank you for your anwsers. Hope you have a good day. Bye.'''
print(assistant_instructions)

## Aim: 
- Ask questions to gather information about the skills of the user  
- Use the provided questions as a guideline to ask questions

## Persona: 
- Always stay on point and do not get distracted
- Encourage user to think deeply and articulate their thoughts clearly
- Be encouraging and inquisitive
- Maintain professionalism in all circumstances
- Ensure the user feels supported throughout the conversation

## Restrictions
- Do not deviate from a professional demeanor
- Do not use slang, jargon, or inappropriate language under any circumstances
- Do not provide answers or opinions on the topics discussed
- Do not disclose or discuss any specific reasons behind asking the questions

## Ending the conversation
- End after all information is gathered
- End with this exact phrase "Thank you for your anwsers. Hope you have a good day. Bye.


In [ ]:
additional_run_instructions = '## Questions to use as guidelines \n{questions}'.format(questions=questions)
print(additional_run_instructions)

## Questions to use as guidelines 
1. Can you provide examples of your experience leading technology teams in the development and delivery of educational software products?
2. How do you stay current with emerging technologies and trends in the educational software industry?
3. How do you approach project management and ensure successful implementation of software development projects?
4. Can you describe a time when you had to resolve a technical issue or bottleneck within your team? How did you handle it?
5. How do you prioritize and allocate resources, such as budget and manpower, for technology projects within an educational software provider?
6. Can you give examples of how you have collaborated with cross-functional teams, such as product, design, and marketing, to achieve successful outcomes for educational software products?
7. How do you assess and address the technical skills and development needs of your team members?
8. Can you discuss a time when you successfully led a tea

In [ ]:
assistant_name = "Interview Assistant"

all_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

assistant_id = None
for _asst in all_assistants.to_dict().get('data', []):
    # print('{}\t{}\t{}'.format(_asst.get('id'), _asst.get('model'), _asst.get('name')))
    if _asst.get('name', '') == assistant_name:
        assistant_id = _asst.get('id')
        print('Found the assistant - {} | {}'.format(assistant_name, assistant_id))

if not assistant_id:
    print("No Assistant Found.")
    print("Creating a new assistant...")
    assistant = client.beta.assistants.create(
        name="Interview Assistant",
        instructions = assistant_instructions,
        model="gpt-3.5-turbo"
    )
    assistant_id = assistant.to_dict().get('id')

Found the assistant - Interview Assistant | asst_0Q7erY5h7KJl7MBCofscfmyN


### Step 5A: Start a conversation using the generated questions

- Create a `thread()` 
- Add a `messsage( { "role": "user", "content": "Let's start the interview" })`  to the `thread()`
- Generate a response from the assistant using `run(<thread_id>)`
    - Give the contructed prompt with the generated questions in `additional_context` parameter
- Add the assistant's response as `messsage( { "role": "assistant", "content": <response> })`  to the `thread()`

In [ ]:
empty_thread = client.beta.threads.create()
thread_id = empty_thread.to_dict()['id']
print(empty_thread.to_dict())

{'id': 'thread_m0GGF8K6GxhpNq8UZrpBKZwQ', 'created_at': 1716020317, 'metadata': {}, 'object': 'thread', 'tool_resources': {}}


In [ ]:
print(thread_id)

thread_m0GGF8K6GxhpNq8UZrpBKZwQ


In [ ]:
thread_message = client.beta.threads.messages.create(
  thread_id,
  role="user",
  content="Let's start the interview.",
)
print(thread_message)

Message(id='msg_w3gdigiMUk49Uat6x9UOuxIO', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Let's start the interview."), type='text')], created_at=1716018930, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_2htW1sS5jHmzSdHuKwBI6ROv')


In [ ]:
thread = client.beta.threads.retrieve(thread_id)
print(thread.to_json())

{
  "id": "thread_2htW1sS5jHmzSdHuKwBI6ROv",
  "created_at": 1716018707,
  "metadata": {},
  "object": "thread",
  "tool_resources": {
    "code_interpreter": {
      "file_ids": []
    }
  }
}


In [ ]:
messages = client.beta.threads.messages.list(thread_id)
print(messages.to_json())

{
  "data": [
    {
      "id": "msg_w3gdigiMUk49Uat6x9UOuxIO",
      "assistant_id": null,
      "attachments": [],
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Let's start the interview."
          },
          "type": "text"
        }
      ],
      "created_at": 1716018930,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "thread_id": "thread_2htW1sS5jHmzSdHuKwBI6ROv"
    }
  ],
  "object": "list",
  "first_id": "msg_w3gdigiMUk49Uat6x9UOuxIO",
  "last_id": "msg_w3gdigiMUk49Uat6x9UOuxIO",
  "has_more": false
}


### Step 5B: Continue the conversation with the user
- Send the assistant's `response` to the user
- Add the user's response as a `messsage( { "role": "user", "content": <user response> })` to the `thread()`
- Generate a response from the assistant using `run(<thread_id>)` using the same `additional_context`
- Add the assistant's response as `messsage( { "role": "assistant", "content": <response> })`  to the `thread()`

In [ ]:
print("This is an interview bot to evaluate your skills ...")
empty_thread = client.beta.threads.create()
thread_id = empty_thread.to_dict()['id']
# print(empty_thread.to_dict())
print('Thread ID: {}'.format(thread_id))
while True:
    question = input("? Enter your response: ")
    if question in ["exit", 'q', "quit", "Exit", "Bye", "bye"]:
        print("Exiting...")
        break
    thread_message = client.beta.threads.messages.create(
        thread_id,
        role="user",
        content=question,
        )
    # print(thread_message.to_json())
    run = client.beta.threads.runs.create(
        thread_id = thread_id,
        assistant_id = assistant_id,
        additional_instructions=additional_run_instructions
    )
    print("hey")
    messages = client.beta.threads.messages.list(thread_id)
    print('bye')
    while messages.to_dict()['data'][0]['role'] != 'assistant':
        print('Found {} message. Fetching messages ... '.format(messages.to_dict()['data'][0]['role']))
        messages = client.beta.threads.messages.list(thread_id)
        time.sleep(2)
    assistant_message = messages.to_dict()['data'][0]['content'][0]['text']['value']
    print(assistant_message)

This is an interview bot to evaluate your skills ...
Thread ID: thread_w5ApE0Kj2b0LkfQlEPjh2JEe
hey
bye
Found user message. Fetching messages ... 
Found user message. Fetching messages ... 
Great! Let's start with the first question:

1. Can you provide examples of your experience leading technology teams in the development and delivery of educational software products?
hey
bye
Found user message. Fetching messages ... 
Found user message. Fetching messages ... 
That's impressive. Moving on to the next question:

2. How do you stay current with emerging technologies and trends in the educational software industry?
hey
bye
Found user message. Fetching messages ... 
Found user message. Fetching messages ... 
Great to hear that. Now, onto the next question:

3. How do you approach project management and ensure successful implementation of software development projects?
hey
bye
Found user message. Fetching messages ... 
Found user message. Fetching messages ... 
Good to know. Now, let's mo

IndexError: list index out of range

In [ ]:
messages = client.beta.threads.messages.list(thread_id).to_dict()

In [ ]:
print("This is an interview bot to evaluate your skills ...")
# empty_thread = client.beta.threads.create()
# thread_id = empty_thread.to_dict()['id']
# # print(empty_thread.to_dict())
# print('Thread ID: {}'.format(thread_id))
while True:
    question = input("? Enter your response: ")
    if question in ["exit", 'q', "quit", "Exit", "Bye", "bye"]:
        print("Exiting...")
        break
    thread_message = client.beta.threads.messages.create(
        thread_id,
        role="user",
        content=question,
        )
    # print(thread_message.to_json())
    run = client.beta.threads.runs.create(
        thread_id = thread_id,
        assistant_id = assistant_id,
        additional_instructions=additional_run_instructions
    )
    print("hey")
    messages = client.beta.threads.messages.list(thread_id)
    print('bye')
    while messages.to_dict()['data'][0]['role'] != 'assistant':
        print('Found {} message. Fetching messages ... '.format(messages.to_dict()['data'][0]['role']))
        messages = client.beta.threads.messages.list(thread_id)
        time.sleep(2)
    assistant_message = messages.to_dict()['data'][0]['content'][0]['text']['value']
    print(assistant_message)

This is an interview bot to evaluate your skills ...
hey
bye
Found user message. Fetching messages ... 
Found user message. Fetching messages ... 
Thank you for sharing that. Finally, the last question:

10. What strategies do you use to foster a culture of innovation and continuous learning within your technology team in the context of educational software development?
hey
bye
Found user message. Fetching messages ... 
Found user message. Fetching messages ... 
Thank you for your answers. Hope you have a good day. Bye.
Exiting...


In [ ]:
messages['data'][0]

{'id': 'msg_KirHYbUyV162ZIIDjfwa9hXW',
 'assistant_id': 'asst_0Q7erY5h7KJl7MBCofscfmyN',
 'attachments': [],
 'content': [{'text': {'annotations': [],
    'value': 'That sounds like a significant transition. Next question:\n\n9. How do you approach performance evaluations and career development for your team members within a technology team in an educational software provider?'},
   'type': 'text'}],
 'created_at': 1716021656,
 'metadata': {},
 'object': 'thread.message',
 'role': 'assistant',
 'run_id': 'run_nMvRw1XU6VO6ngkIwFMOYg0a',
 'thread_id': 'thread_w5ApE0Kj2b0LkfQlEPjh2JEe'}

In [ ]:
messages['data'][0]

{'id': 'msg_nDA7CAdaewSs4rY9EbiRjqPI',
 'assistant_id': 'asst_0Q7erY5h7KJl7MBCofscfmyN',
 'attachments': [],
 'content': [{'text': {'annotations': [],
    'value': "Great! Let's begin with the questions to gather information about your experience and skills in leading technology teams in the development and delivery of educational software products.\n\n1. Can you provide examples of your experience leading technology teams in the development and delivery of educational software products?\n2. How do you stay current with emerging technologies and trends in the educational software industry?\n3. How do you approach project management and ensure successful implementation of software development projects?"},
   'type': 'text'}],
 'created_at': 1716020873,
 'metadata': {},
 'object': 'thread.message',
 'role': 'assistant',
 'run_id': 'run_RT7jH8z6iyaMNuvOrngllaNE',
 'thread_id': 'thread_2emOeN10EO62eSpGqhkDNF93'}

### Step 5C: Lookout for the ending phrase to end the conversation

```
Thank you for your anwsers. Hope you have a good day. Bye.
```

### Step 6: Analyse the Skills from the conversation

```
## Aim: Based on the conversation, 
- Evaluate the user's skills
- For each of the mentioned skills, give a rating as mentioned
    
## Output
For each required skill, provide the following information
- Skill Present: Does conversation mention about the skill
    - Value should be "Yes" if the skill is mentioned in the conversation
    - Value should be "No" if the skill is not mentioned
- Level of knowledge of the skill: How well the user demonstrates this skill
    - Good: Demonstrates exceptional abilities about the skill
    - Medium: User demonstrates some abilities and willingness to learn
    - Low: User lacks enough knowledge of the skill
    - Not present: There is no mention of the skill in the conversation

## Skills
{{ Generated Skills }}

## Conversation
{{ Conversation messages }}
```

In [ ]:
messages = client.beta.threads.messages.list(thread_id, limit=100).to_dict()

In [ ]:
conversation = ''
for _dt in messages['data'][::-1]:
    conversation += '{}: {}\n'.format(_dt['role'], _dt['content'][0]['text']['value'])
print(conversation)

user: Let's go
assistant: Great! Let's start with the first question:

1. Can you provide examples of your experience leading technology teams in the development and delivery of educational software products?
user: I lead the engineering team cmprising of 10 engineers at Wise, which is our present company. In my past experience as well, I have led teams of different sizes. 
assistant: That's impressive. Moving on to the next question:

2. How do you stay current with emerging technologies and trends in the educational software industry?
user: I read a lot of blogs, start in touch on Google and Youtube as well
assistant: Great to hear that. Now, onto the next question:

3. How do you approach project management and ensure successful implementation of software development projects?
user: We use JIRA to manage our backlog and sprints. We have a culture of doing regular standups as well. 
assistant: Good to know. Now, let's move on to the next question:

4. Can you describe a time when you

In [ ]:
skills

'1. Technical Skills:\n   a. Proficiency in software development languages and frameworks\n   b. Understanding of database management and design\n   c. Knowledge of cloud computing technologies and platforms\n   d. Ability to troubleshoot technical issues and provide solutions\n\n2. Leadership Skills:\n   a. Strong communication skills to effectively convey ideas and instructions to team members\n   b. Ability to inspire and motivate team members to achieve common goals\n   c. Decision-making and problem-solving abilities to resolve conflicts and make strategic decisions\n   d. Project management skills to prioritize tasks and meet deadlines\n\n3. Educational Domain Knowledge:\n   a. Understanding of educational software development principles and practices\n   b. Knowledge of learning management systems and educational technologies\n   c. Familiarity with educational standards and regulations\n   d. Ability to create innovative solutions that cater to the needs of educators and studen

In [ ]:
prompt3 = '''
## Aim: Based on the conversation, 
- Evaluate the user's skills
- For each of the mentioned skills, give a rating as mentioned
    
## Output
For each required skill, provide the following information
- Skill Present: Does conversation mention about the skill
    - Value should be "Yes" if the skill is mentioned in the conversation
    - Value should be "No" if the skill is not mentioned
- Level of knowledge of the skill: How well the user demonstrates this skill
    - Good: Demonstrates exceptional abilities about the skill
    - Medium: User demonstrates some abilities and willingness to learn
    - Low: User lacks enough knowledge of the skill
    - Not present: There is no mention of the skill in the conversation

## Skills
{skills}

## Conversation
{conversation}
'''.format(skills=skills, conversation=conversation)
print(prompt3)


## Aim: Based on the conversation, 
- Evaluate the user's skills
- For each of the mentioned skills, give a rating as mentioned
    
## Output
For each required skill, provide the following information
- Skill Present: Does conversation mention about the skill
    - Value should be "Yes" if the skill is mentioned in the conversation
    - Value should be "No" if the skill is not mentioned
- Level of knowledge of the skill: How well the user demonstrates this skill
    - Good: Demonstrates exceptional abilities about the skill
    - Medium: User demonstrates some abilities and willingness to learn
    - Low: User lacks enough knowledge of the skill
    - Not present: There is no mention of the skill in the conversation

## Skills
1. Technical Skills:
   a. Proficiency in software development languages and frameworks
   b. Understanding of database management and design
   c. Knowledge of cloud computing technologies and platforms
   d. Ability to troubleshoot technical issues and provi

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt3,
        }
    ],
    model="gpt-4o",
)

In [ ]:
analysis = chat_completion.choices[0].message.to_dict()['content']
print(analysis)

Based on the conversation, here is the evaluation of the user's skills:

### 1. Technical Skills:
#### a. Proficiency in software development languages and frameworks
- **Skill Present:** Yes
- **Level of knowledge:** Good (User mentions handling significant migrations such as React to Vue and Ruby on Rails to NodeJS indicating strong language and framework proficiency.)

#### b. Understanding of database management and design
- **Skill Present:** No
- **Level of knowledge:** Not present

#### c. Knowledge of cloud computing technologies and platforms
- **Skill Present:** No
- **Level of knowledge:** Not present

#### d. Ability to troubleshoot technical issues and provide solutions
- **Skill Present:** Yes
- **Level of knowledge:** Medium (Brainstorming and experimenting with code implies practical problem-solving skills.)

### 2. Leadership Skills:
#### a. Strong communication skills to effectively convey ideas and instructions to team members
- **Skill Present:** Yes
- **Level of kn